# Sports14 Text/Image Feature Extraction

In [1]:

import os
import numpy as np
import pandas as pd
from transformers.image_utils import load_image

In [2]:
os.chdir('/data/sports')
os.getcwd()

'C:\\Users\\z\\PycharmProjects\\MMRec\\data\\Sports14'

## Load text data

In [3]:
i_id, desc_str = 'itemID', 'description'

file_path = './'
file_name = 'meta-sports14.csv'

meta_file = os.path.join(file_path, file_name)

df = pd.read_csv(meta_file)
df.sort_values(by=[i_id], inplace=True)

print('data loaded!')
print(f'shape: {df.shape}')

df[:3]

data loaded!
shape: (18357, 10)


,itemID,asin,title,price,imUrl,related,brand,categories,salesRank,description
0,0,1881509818,Ghost Inc Glock Armorers Tool 3/32 Punch,9.99,http://ecx.images-amazon.com/images/I/21iMxsyD...,"{'also_bought': ['B000U3YWEM', 'B000U401J6', '...",Ghost,"[['Sports & Outdoors', 'Hunting & Fishing', 'H...",{'Sports &amp; Outdoors': 172909},Ghost Armorer Tool (1). The GAT is made with a...
1,1,2094869245,5 LED Bicycle Rear Tail Red Bike Torch Laser B...,8.26,http://ecx.images-amazon.com/images/I/51RtwnJw...,"{'also_bought': ['B0081O93N2', 'B00EYTCHJA', '...",NaN,"[['Sports & Outdoors', 'Cycling', 'Lights & Re...",{'Sports &amp; Outdoors': 14293},This newly-designed Laser tail light can emit ...
2,2,7245456259,Black Mountain Products Single Resistance Band...,10.49,http://ecx.images-amazon.com/images/I/411Ikpf1...,"{'also_bought': ['B00DDBS2JE', 'B00H1KNHE8', '...",Black Mountain,"[['Sports & Outdoors', 'Exercise & Fitness', '...",{'Sports &amp; Outdoors': 1010},Black Mountain Products single resistance band...


In [4]:

# sentences: title + brand + category + description | All have title + description

title_na_df = df[df['title'].isnull()]
print(title_na_df.shape)

desc_na_df = df[df['description'].isnull()]
print(desc_na_df.shape)

na_df = df[df['description'].isnull() & df['title'].isnull()]
print(na_df.shape)

na3_df = df[df['description'].isnull() & df['title'].isnull() & df['brand'].isnull()]
print(na3_df.shape)

na4_df = df[df['description'].isnull() & df['title'].isnull() & df['brand'].isnull() & df['categories'].isnull()]
print(na4_df.shape)

(91, 10)
(2659, 10)
(40, 10)
(40, 10)
(0, 10)


In [5]:

df[desc_str] = df[desc_str].fillna(" ")
df['title'] = df['title'].fillna(" ")
df['brand'] = df['brand'].fillna(" ")
df['categories'] = df['categories'].fillna(" ")


In [6]:
sentences = []
for i, row in df.iterrows():
    sen = row['title'] + ' ' + row['brand'] + ' '
    cates = eval(row['categories'])
    if isinstance(cates, list):
        for c in cates[0]:
            sen = sen + c + ' '
    sen += row[desc_str]
    sen = sen.replace('\n', ' ')

    sentences.append(sen)

sentences[:10]

['Ghost Inc Glock Armorers Tool 3/32 Punch Ghost Sports & Outdoors Hunting & Fishing Hunting Gun Maintenance Gunsmithing Tools Ghost Armorer Tool (1). The GAT is made with a spring steel punch. The diameter is 3/32 of an inch or 2.5mm, this is the same as the OEM tool size. The difference is you will be able to press harder without bending the shaft of this punch. Just a better tool to work on your Glock with.',
 '5 LED Bicycle Rear Tail Red Bike Torch Laser Beam Lamp Light   Sports & Outdoors Cycling Lights & Reflectors Taillights This newly-designed Laser tail light can emit two parallel lines, to form a virtual lane together with the moving of bicycle on the road. LED flash light and  two lines not only enhance the waring effect strongly and greatly but also improve the safety of night riding.',
 'Black Mountain Products Single Resistance Band - Door Anchor and Starter Guide Included Black Mountain Sports & Outdoors Exercise & Fitness Accessories Exercise Bands Black Mountain Produc

In [7]:

course_list = df[i_id].tolist()
#sentences = df[desc_str].tolist()

assert course_list[-1] == len(course_list) - 1

In [12]:
# should `pip install sentence_transformers` first
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

sentence_embeddings = model.encode(sentences)
print('text encoded!')

assert sentence_embeddings.shape[0] == df.shape[0]
np.save(os.path.join(file_path, 'text_feat.npy'), sentence_embeddings)
print('done!')


C:\Users\z\miniconda3\envs\MMRec\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name all-MiniLM-L6-v2. Creating a new one with mean pooling.


text encoded!
done!


In [13]:
sentence_embeddings[:10]

array([[-0.37406778,  0.09901473, -0.05774754, ..., -0.30028063,
         0.15247405,  0.21734801],
       [ 0.02025855,  0.00165912, -0.09402552, ...,  0.10188628,
         0.07298157,  0.09271482],
       [-0.2945816 ,  0.13180391, -0.00647214, ..., -0.47098812,
         0.09913322,  0.1213255 ],
       ...,
       [-0.13677554,  0.09039376, -0.0074961 , ...,  0.04929856,
        -0.05276965,  0.13957648],
       [ 0.1188093 ,  0.08956951, -0.10169142, ...,  0.02228665,
         0.11932227,  0.07714809],
       [-0.24403696,  0.14211708, -0.0363807 , ..., -0.20753446,
        -0.00919836,  0.0718814 ]], dtype=float32)

In [14]:
load_txt_feat = np.load('text_feat.npy', allow_pickle=True)
print(load_txt_feat.shape)
load_txt_feat[:10]

(18357, 384)


array([[-0.37406778,  0.09901473, -0.05774754, ..., -0.30028063,
         0.15247405,  0.21734801],
       [ 0.02025855,  0.00165912, -0.09402552, ...,  0.10188628,
         0.07298157,  0.09271482],
       [-0.2945816 ,  0.13180391, -0.00647214, ..., -0.47098812,
         0.09913322,  0.1213255 ],
       ...,
       [-0.13677554,  0.09039376, -0.0074961 , ...,  0.04929856,
        -0.05276965,  0.13957648],
       [ 0.1188093 ,  0.08956951, -0.10169142, ...,  0.02228665,
         0.11932227,  0.07714809],
       [-0.24403696,  0.14211708, -0.0363807 , ..., -0.20753446,
        -0.00919836,  0.0718814 ]], dtype=float32)

# Image encoder (V0)，following LATTICE, averaging over for missed items

In [15]:
df[:5]

itemID        asin  \
0       0  1881509818   
1       1  2094869245   
2       2  7245456259   
3       3  7245456313   
4       4  B000002NUS   

                                                                                                                          title  \
0                                                                                      Ghost Inc Glock Armorers Tool 3/32 Punch   
1                                                                  5 LED Bicycle Rear Tail Red Bike Torch Laser Beam Lamp Light   
2                                       Black Mountain Products Single Resistance Band - Door Anchor and Starter Guide Included   
3  Black Mountain Products Resistance Band Set with Door Anchor, Ankle Strap, Exercise Chart, and Resistance Band Carrying Case   
4                                                                        Outers Universal 32-Piece Blow Molded Gun Cleaning Kit   

   price                                                          imUrl  \
0   9.99  http://ecx.images-amazon.com/images/I/21iMxsyDBRL._SX300_.jpg   
1   8.26  http://ecx.images-amazon.com/images/I/51RtwnJwtBL._SY300_.jpg   
2  10.49  http://ecx.images-amazon.com/images/I/411Ikpf122L._SY300_.jpg   
3  32.99  http://ecx.images-amazon.com/images/I/51FdHlZSi7L._SY300_.jpg   
4  21.99  http://ecx.images-amazon.com/images/I/510GjWgdTVL._SY300_.jpg   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 related  \
0                                                                        {'also_bought': ['B000U3YWEM', 'B000U401J6', 'B004JLY2GE', 'B004NGP8X6', 'B003WG95H8', 'B006MYLE9O', 'B0065PJY60', 'B008VYYD6Y', 'B00A1WEMRE', 'B004Z0LQYA', 'B0081JJVUC', 'B001AT6S7O', 'B00FGL97CA', 'B000J4HN9I', 'B004ERKCIA'

In [16]:
import array

def readImageFeatures(path):
  f = open(path, 'rb')
  while True:
    asin = f.read(10).decode('UTF-8')
    if asin == '': break
    a = array.array('f')
    a.fromfile(f, 4096)
    yield asin, a.tolist()

In [17]:

img_data = readImageFeatures("image_features_Sports_and_Outdoors.b")
item2id = dict(zip(df['asin'], df['itemID']))

feats = {}
avg = []
for d in img_data:
    if d[0] in item2id:
        feats[int(item2id[d[0]])] = d[1]
        avg.append(d[1])
avg = np.array(avg).mean(0).tolist()

ret = []
non_no = []
for i in range(len(item2id)):
    if i in feats:
        ret.append(feats[i])
    else:
        non_no.append(i)
        ret.append(avg)

print('# of items not in processed image features:', len(non_no))
assert len(ret) == len(item2id)
np.save('image_feat.npy', np.array(ret))
np.savetxt("missed_img_itemIDs.csv", non_no, delimiter =",", fmt ='%d')
print('done!')

# of items not in processed image features: 180
done!


In [18]:
load_image_feat = np.load('image_feat.npy', allow_pickle=True)
print(load_image_feat.shape)
load_image_feat[:10]

(18357, 4096)


array([[0.        , 5.27110004, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 2.98839998, 0.41280001,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.72619998,
        0.        ],
       ...,
       [0.        , 0.67030001, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 1.00390005, 3.91120005,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        2.92260003]])